# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096905


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:36,  3.43s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:05,  2.43s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:45,  1.75s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.27s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:09<00:00,  7.66it/s]

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  7.84it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  8.92it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  8.15it/s]

not-logged-in-461bf2f260fc99e4866e    0.000974
not-logged-in-589ba43a9fa7db595cc3    0.001518
Alexander00                           0.001980
TateK                                 0.001719
jnarayanbvg                           0.000260
not-logged-in-0707d430a0610af06bb6    0.015211
jaksonA                               0.068336
not-logged-in-ed24b424692bed06d715    0.001025
rmz4160                               0.000392
SciKid10                              0.004151
jstucky                               0.000426
not-logged-in-74c292c98a962cc17454    0.022785
not-logged-in-3a93234a2311f6df1afd    0.000476
not-logged-in-15c63a3909f8669e205d    0.045684
pangeli5                              0.000420
alexmnelson18                         0.000313
moasda                                0.000564
farheenk88                            0.000485
Lavadude                              0.036335
w7250369                              0.001744
Fla                                   0.002708
Jackson2802  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())